# FrozenLake8x8

**S**: Starting position  
**F**: Frozen platform  
**H**: Hole  
**G**: Goal  
  
actions: [**LEFT**, **DOWN**, **RIGHT**, **UP**]
  
The goal during this game is to reach the tile 'G' starting from the tile 'S' without falling into a hole ('H').  
The environment is -by default- slippery. This means that we cannot guarantee that a particular action will move the agent to the direction it says it will. The environment doesn't allow the agent to go off the map so a step that would cause exiting will have no effect.




In [1]:
import gym
import numpy as np

In [2]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

PROB = 0
S_PRIME = 1
REWARD = 2

## State transition nondeterminism

By playing a quick game choosing random actions exclusively and printing the info variable that the environment returns after a taken step we can now see that we're only having 1/3 chance of going to the direction we selected.

In [3]:
env = gym.make('FrozenLake8x8-v0', is_slippery=True) # is_slippery by default True
env.reset()
env.render()

for i in range(0, 3):
    random_action = env.action_space.sample()        #choose a random action
    new_s, r, done, info = env.step(random_action)
    print(info)
    env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
{'prob': 0.3333333333333333}
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
{'prob': 0.3333333333333333}
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
{'prob': 0.3333333333333333}
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


We can further analize each step by looking into the *env.P[state][action]* data structure which will reveal how the state transitions work.    
Now, we can clearly see that if we choose a specific direction we have only 1/3 chance that the agent will follow our command the way we would like. The agent might go to 90 degrees left or 90 degrees right to the preferred direction (with probability of 1/3 each) and never the opposite way.

In [4]:
env.P[9][UP]
for info in env.P[9][UP]:
    print(info)

(0.3333333333333333, 10, 0.0, False)
(0.3333333333333333, 1, 0.0, False)
(0.3333333333333333, 8, 0.0, False)


## Policy iteration

In [5]:
def expected_state_action_value(env, state_values, s, a, dsc_rate):
    q = 0
    for info in env.P[s][a]:
        q += info[PROB]*(info[REWARD] + dsc_rate * state_values[info[S_PRIME]])
    
    return q

In [6]:
def policy_evaluation(env, policy, theta, dsc_rate):
    state_values = np.zeros(64)
    delta = np.inf
    
    while delta >= theta:
        delta = 0
        
        for s in range(64):
            v = state_values[s]
            new_v = 0
            for a in range(4):
                new_v += policy[s][a] * expected_state_action_value(env, state_values, s, a, dsc_rate)
                
            state_values[s] = new_v
            delta = max(delta, np.abs(v - new_v))
    
    return state_values             

In [7]:
def policy_improvement(env, policy, state_values, dsc_rate):
    new_policy = np.zeros((64, 4))  
    q = np.zeros((64, 4))           # state-action values
    
    for s in range(64):
        for a in range(4):
            q[s][a] = expected_state_action_value(env, state_values, s, a, dsc_rate)
        a_index = q[s].argmax()       
        new_policy[s] = np.zeros(4)
        new_policy[s][a_index] = 1
           
    return new_policy

In [8]:
def policy_iteration(env, theta=0.00001, dsc_rate=0.99):
    env.reset()
    state_values = np.zeros(64)
    policy = np.array([[0.25, 0.25, 0.25, 0.25],] * 64)
    
    policy_stable = False
    while not policy_stable:
        state_values = policy_evaluation(env, policy, theta, dsc_rate)
        new_policy = policy_improvement(env, policy, state_values, dsc_rate)
        if (policy == new_policy).all():
            policy_stable = True
            
        policy = new_policy
    
    return policy    

In [9]:
%%timeit
policy_iteration(env)

743 ms ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
policy = policy_iteration(env)

## Evaluation of policy iteration

In [11]:
def play_frozen_lake(env, policy, amount_of_games = 1):
    won_games = 0
    for _ in range(amount_of_games):
        env.reset()
        s = 0
        while True:
            action = policy[s].argmax()
            new_s, r, done, info = env.step(action)
            if done:
                if r == 1.0:
                    # we successfully reached our goal
                    won_games += 1
                break
            s = new_s            
    
    win_ratio = won_games / amount_of_games
    print("We won {:.2%} of the games".format(win_ratio))

In [12]:
play_frozen_lake(env, policy, 1000)

We won 87.30% of the games


We could see that the policy iteration method converges to the (near) optimal policy in 743ms, averagely.
Following the policy we computed we win the frozen lake game in 87.3% of the times.

# Value iteration

In [13]:
def value_iteration(env, theta=0.00001, dsc_rate=0.99):
    env.reset()
    state_values = np.zeros(64)
    policy = np.array([[0.25, 0.25, 0.25, 0.25],] * 64)
    delta = np.inf
    
    while delta >= theta:
        delta = 0
        
        for s in range(64):
            v = state_values[s]
            candidate_v = np.zeros(4)
                       
            for a in range(4):
                candidate_v[a] = expected_state_action_value(env, state_values, s, a, dsc_rate)
            
            new_v = candidate_v.max()
            delta = max(delta, np.abs(v - new_v))            
            state_values[s] = new_v
    
    policy = policy_improvement(env, policy, state_values, dsc_rate)
    return policy    

## Evaluation of value iteration

In [14]:
%%timeit
value_iteration(env)

318 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
policy = value_iteration(env)

In [16]:
play_frozen_lake(env, policy, 1000)

We won 86.80% of the games


Value iteration seemingly performed overall better:  
The policy was found in shorter time and the win ratio is basically the same.

# Further evaluations

## Win ratio in case of deterministic environment
We would expect 100% win ratio if we get rid of the influence of the slipperiness of the field.

In [17]:
env = gym.make('FrozenLake8x8-v0', is_slippery=False)

policy1 = policy_iteration(env)
policy2 = value_iteration(env)

play_frozen_lake(env, policy1, 1000)
play_frozen_lake(env, policy2, 1000)

We won 100.00% of the games
We won 100.00% of the games
